### Part 1, 2 & 3

## Importing/Installing Libraries

In [2]:
#conda install -c anaconda beautifulsoup4
import pandas as pd
from bs4 import BeautifulSoup
import requests

## Obtaining the data from Wikipedia with request.get and Beautiful Soup

In [3]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

## Creating Lists for each of the Columns to store the data scraped 

In [4]:
postalCodeList = []
boroughList = []
neighborhoodList = []
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

## Creating a dataframe using the information stored in the list above

In [5]:
df = pd.DataFrame({"PostalCode": postalCodeList, "Borough": boroughList, "Neighborhood": neighborhoodList})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Dropping the "Not assigned" from the Borough column

In [6]:
df = df[df.Borough != "Not assigned"].reset_index(drop=True)

## Group the data by the PostalCode and then Borough

In [7]:
df = df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


## Now to Create the df that the example provided.

In [8]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
fixed_df = pd.DataFrame(columns=column_names)
postal_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for code in postal_list:
    fixed_df = fixed_df.append(df[df["PostalCode"] == code], ignore_index=True)
    
fixed_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens, Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


## Shape of the dataframe

In [9]:
df.shape

(103, 3)

## Importing the coordinate data using Panda

In [10]:
coordinate = pd.read_csv('http://cocl.us/Geospatial_data')
coordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Need to change the column name before able to merge

In [11]:
coordinate.rename(columns = {"Postal Code": "PostalCode"}, inplace=True)
coordinate.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge the two Dataframe base on whichever shares the same PostalCode

In [12]:
new_df = df.merge(coordinate, on = "PostalCode", how = "left")
new_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


## Create the Dataframe shown as example

In [13]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
fixed_df = pd.DataFrame(columns=column_names)
for code in postal_list:
    fixed_df = fixed_df.append(new_df[new_df["PostalCode"] == code], ignore_index = True)
    
fixed_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


## More libraries

In [36]:
#conda install -c conda-forge geopy 
from geopy.geocoders import Nominatim
import folium
import json 
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Prepping Steps for Map of Toronto

In [15]:
geolocator = Nominatim(user_agent = "SD125")
location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude = location.longitude

In [16]:
borough_names = list(new_df.Borough.unique())

borough_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_toronto.append(x)
        
borough_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [17]:
new_df = new_df[new_df['Borough'].isin(borough_toronto)].reset_index(drop=True)
print(new_df.shape)
new_df.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [18]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Borough'], new_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 4,
        popup = label,
        color = 'red',
        fill = True,
        fill_color = ' #e74c3c',
        fill_opacity = 0.7).add_to(map_toronto)  
    
map_toronto

## Using FourSquare API to obtain data

In [21]:
CLIENT_ID = 'NTRQHQBQPAVHKGAYHYEBTHQFZSG3CJVYJWNL0544FSX032TV' 
CLIENT_SECRET = 'YX0Q2FHJXTGZYBEQCALXI1J0QEUX1Z2AETSQX3FQCT2F1VNI' 
VERSION = '20180605'

In [22]:
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['PostalCode'], new_df['Borough'], new_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

## Turning the obtained data into a dataframe

In [23]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
venues_df.head()

(1698, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


## One hot encoding and editting Dataframe

In [24]:
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1698, 228)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 228)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,...,0.000000,0.000000,0.02381,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.023810
2,M4L,East Toronto,"The Beaches West, India Bazaar",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,M4M,East Toronto,Studio District,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048780,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.024390,0.00,0.024390
4,M4N,Central Toronto,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,M4P,Central Toronto,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,M4R,Central Toronto,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.055556
7,M4S,Central Toronto,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.029412,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.066667,0.000000,0.00,0.000000


## Creating Columns according to the number of top venues

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Neighborhood,Health Food Store,Pub,Trail,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Liquor Store
2,M4L,East Toronto,"The Beaches West, India Bazaar",Sandwich Place,Sushi Restaurant,Fish & Chips Shop,Pub,Fast Food Restaurant,Brewery,Ice Cream Shop,Food & Drink Shop,Liquor Store,Pizza Place
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Brewery,Italian Restaurant,Bakery,American Restaurant,Sandwich Place,Cheese Shop,Pet Store
4,M4N,Central Toronto,Lawrence Park,Park,Lawyer,Bus Line,Swim School,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
5,M4P,Central Toronto,Davisville North,Department Store,Hotel,Park,Breakfast Spot,Gym,Sandwich Place,Food & Drink Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
6,M4R,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Café,Spa,Diner,Sporting Goods Shop,Fast Food Restaurant,Salon / Barbershop,Dessert Shop
7,M4S,Central Toronto,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Gym,Italian Restaurant,Café,Coffee Shop,Pharmacy,Brewery
8,M4T,Central Toronto,"Moore Park, Summerhill East",Playground,Tennis Court,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Pub,Coffee Shop,Athletics & Sports,Supermarket,Vietnamese Restaurant,Sushi Restaurant,Sports Bar,American Restaurant,Light Rail Station,Pizza Place


## K-Mean Cluster

In [31]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([3, 3, 3, 3, 0, 3, 3, 3, 1, 3], dtype=int32)

In [33]:
toronto_merged = new_df.copy()
toronto_merged["Cluster Labels"] = kmeans.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on = "PostalCode")

print(toronto_merged.shape)
toronto_merged.head() 

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Neighborhood,Health Food Store,Pub,Trail,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Liquor Store
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3,Sandwich Place,Sushi Restaurant,Fish & Chips Shop,Pub,Fast Food Restaurant,Brewery,Ice Cream Shop,Food & Drink Shop,Liquor Store,Pizza Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Café,Coffee Shop,Gastropub,Brewery,Italian Restaurant,Bakery,American Restaurant,Sandwich Place,Cheese Shop,Pet Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Lawyer,Bus Line,Swim School,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [34]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Lawyer,Bus Line,Swim School,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Trail,Playground,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,1,Playground,Tennis Court,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,2,Garden,Home Service,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Neighborhood,Health Food Store,Pub,Trail,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Yoga Studio
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,3,Sandwich Place,Café,Coffee Shop,Middle Eastern Restaurant,Pharmacy,Flower Shop,Indian Restaurant,Cheese Shop,Liquor Store,BBQ Joint
25,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049,3,Café,Bakery,Bookstore,Japanese Restaurant,Restaurant,Bar,College Arts Building,Coffee Shop,Chinese Restaurant,Dessert Shop
26,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,3,Vietnamese Restaurant,Café,Bar,Chinese Restaurant,Dumpling Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Bakery,Dessert Shop
27,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,3,Airport Service,Airport Terminal,Airport Lounge,Sculpture Garden,Boat or Ferry,Boutique,Harbor / Marina,Airport Food Court,Airport,Bar
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,3,Coffee Shop,Café,Cocktail Bar,Italian Restaurant,Seafood Restaurant,Beer Bar,Japanese Restaurant,Restaurant,Hotel,Breakfast Spot


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1 (Parks and other recreation facilities)

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,0,Park,Lawyer,Bus Line,Swim School,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
10,Downtown Toronto,0,Park,Trail,Playground,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


## Cluster 2 (Play Ground and Tennis Court)

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Playground,Tennis Court,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


## Cluster 3 (Garden and Home Service)

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Garden,Home Service,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


## Cluster 4 (Food Outlets and Dinning)

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3,Neighborhood,Health Food Store,Pub,Trail,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Yoga Studio
24,Central Toronto,3,Sandwich Place,Café,Coffee Shop,Middle Eastern Restaurant,Pharmacy,Flower Shop,Indian Restaurant,Cheese Shop,Liquor Store,BBQ Joint
25,Downtown Toronto,3,Café,Bakery,Bookstore,Japanese Restaurant,Restaurant,Bar,College Arts Building,Coffee Shop,Chinese Restaurant,Dessert Shop
26,Downtown Toronto,3,Vietnamese Restaurant,Café,Bar,Chinese Restaurant,Dumpling Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Bakery,Dessert Shop
27,Downtown Toronto,3,Airport Service,Airport Terminal,Airport Lounge,Sculpture Garden,Boat or Ferry,Boutique,Harbor / Marina,Airport Food Court,Airport,Bar
28,Downtown Toronto,3,Coffee Shop,Café,Cocktail Bar,Italian Restaurant,Seafood Restaurant,Beer Bar,Japanese Restaurant,Restaurant,Hotel,Breakfast Spot
29,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Steakhouse,Bar,Gastropub,Gym,Seafood Restaurant,Hotel,Asian Restaurant
30,Downtown Toronto,3,Grocery Store,Café,Park,Baby Store,Coffee Shop,Restaurant,Diner,Italian Restaurant,Candy Store,Gas Station
31,West Toronto,3,Bakery,Pharmacy,Furniture / Home Store,Brazilian Restaurant,Fast Food Restaurant,Café,Bar,Bank,Supermarket,Middle Eastern Restaurant
32,West Toronto,3,Bar,Asian Restaurant,Restaurant,Coffee Shop,Vietnamese Restaurant,Pizza Place,Men's Store,Café,Yoga Studio,Mexican Restaurant


## Cluster 5 (Jewelry Store and Trail)

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Jewelry Store,Trail,Sushi Restaurant,Mexican Restaurant,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Cluster 4 contains the most neighborhoods while the rest of the clusters have at most only 2. 